In [3]:
from caveclient import CAVEclient
import os
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network

import sys
sys.path.append('utils')
from FANC_utils import filter_syn_df, compute_connectivity_matrix, hl_nt_dict, sort_partial_match

In [2]:
# Initialize CAVEclient
# This step requires an authentication token, see https://caveclient.readthedocs.io/en/latest/guide/authentication.html
client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 840

In [4]:
# Specify 9A IDs 
# client.materialize.version = 840
# timestamp = '2024-01-17T08:10:01.179472'
source_ids = [648518346496946148, # chief_9A
              648518346479847574, # 9A_1
              648518346479837078, # 9A_2
              648518346498002535, # 9A_3
              648518346479879156, # 9A_4
              648518346467364359, # 9A_5
              648518346486716621] # 9A_6

In [6]:
# Query input synapses
syn_df = client.materialize.synapse_query(post_ids=source_ids)

In [7]:
# Remove connections less than syn_thresh from syn_df 
syn_thresh = 3;
syn_df_filt = filter_syn_df(syn_df, syn_thresh)

In [8]:
# Get neuron information
neuron_info_df = client.materialize.query_table('neuron_information')

In [9]:
# Get target IDs
tag = 'descending'
target_ids = neuron_info_df.pt_root_id[neuron_info_df.tag==tag].tolist()

In [11]:
# Find synapses with descending neurons
ix_to_include = []
for ix, i in syn_df_filt.iterrows():
    if syn_df_filt.pre_pt_root_id[ix] in target_ids: 
        ix_to_include.append(ix)

In [12]:
# Remove connections with neurons other than descending neurons
syn_df_filt_filt = syn_df_filt.loc[ix_to_include]

In [16]:
# Sum synapses for each partner neuron
syn_per_conn = syn_df_filt_filt.groupby(['pre_pt_root_id','post_pt_root_id'])['post_pt_root_id'].count().reset_index(name="number_of_synapses").sort_values(by=["number_of_synapses"]).sort_values(by=['number_of_synapses'], ascending=False) #.reset_index(drop=True)
syn_per_conn 

,pre_pt_root_id,post_pt_root_id,number_of_synapses
2,648518346459693060,648518346479837078,162
8,648518346476980936,648518346479837078,129
54,648518346496855192,648518346479837078,103
64,648518346503742369,648518346479837078,87
14,648518346478690132,648518346479837078,75
...,...,...,...
56,648518346497743463,648518346479837078,3
34,648518346487779095,648518346496946148,3
69,648518346514244039,648518346479837078,3
62,648518346502341799,648518346479837078,3


In [17]:
target_ids = syn_per_conn.pre_pt_root_id.unique().tolist()
source_ids = syn_per_conn.post_pt_root_id.unique().tolist()

In [18]:
# Build network graph
neuron_net = nx.DiGraph() 

# Add nodes; have to be strings for pyvis
target_names = [str(i) for i in target_ids]
source_names = [str(i) for i in source_ids]
neuron_net.add_nodes_from(target_names)
neuron_net.add_nodes_from(source_names)

In [19]:
# Add edges
for ix, i in syn_per_conn.iterrows(): 
    neuron_net.add_edge(str(syn_per_conn.pre_pt_root_id[ix]), # Has to be string for pyvis
                        str(syn_per_conn.post_pt_root_id[ix]), 
                        weight=int(syn_per_conn.number_of_synapses[ix])/10,
                        width=int(syn_per_conn.number_of_synapses[ix])/10,
                        title=str(syn_per_conn.number_of_synapses[ix]))   

In [20]:
# Generate html file
net = Network(width='80%', height='1000px')
net.from_nx(neuron_net)
for i in net.nodes:
    i.update({'physics': False})

net.show('DN_9A_connectivity.html', notebook=False)

DN_9A_connectivity.html
